In [4]:
import numpy as np
import pandas as pd

# Importing data
test = pd.read_csv('dataset/test.csv')
train = pd.read_csv('dataset/train.csv')

y_feature = 'SalePrice'

## Cleaning data:
- Categorical Data: assuming that it is a normal distirbutation, break it in to z's = (X-mean)/std
- Fill the missing data with KNN algorithem

In [24]:
# Droping All the columns with more than half of rows nans 
# simply because they might not be as important as the other columns
train.dropna(thresh=train.shape[0] / 2, axis=1, inplace=True)

# Getting the features with missing values
# Divifing the dataset into categorical and numerical
categorical_data = train.select_dtypes('object')
numerical_data = train.select_dtypes(['float64', 'int64'])

# Get the columns with missing data
missing_categorical_data = categorical_data.columns[categorical_data.isna().any()].tolist()
missing_numerical_data = numerical_data.columns[numerical_data.isna().any()].tolist()
missing_categorical_data

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [40]:
""" Converts the categorical data based on the SalePrice column """
def getFeatureInfo(feature):
    #  Instantiate two lists to store the information
    n_list = list(train[feature].unique())
    n_dict = {}

    for para in n_list:
        # Initiate an empty list
        d = {}
        
        # Set the mean and std
        d['mean'] = train[train[feature] == para][y_feature].mean()
        d['std'] = train[train[feature] == para][y_feature].std()
        d['mode'] = train[train[feature] == para][y_feature].mode().mean()
        # Add the info
        n_dict[para] = d
    
    return n_dict

def convert(feature):
    dic = getFeatureInfo(feature)
    print(dic)
    newFeature = feature + "converted"
    l = []
    
    for i in range(0, train.shape[0]):
        mode = dic[train.iloc[i][feature]]['mode']
        print(train.iloc[i][feature].isnull())
        if train.iloc[i][feature] == np.nan:
            l.append(mode)
        else:
            mean = dic[train.iloc[i][feature]]['mean']
            std = dic[train.iloc[i][feature]]['std']
            y = train.iloc[i][y_feature]
            l.append((y - mean) / std)
    
    train[newFeature] = l
    
feature = 'BsmtCond'

convert(feature)
train[feature + "converted"].isna().sum()

{'TA': {'mean': 183632.6209000763, 'std': 79515.38844895519, 'mode': 140000.0}, 'Gd': {'mean': 213599.9076923077, 'std': 72663.15502580276, 'mode': 169250.0}, nan: {'mean': nan, 'std': nan, 'mode': nan}, 'Fa': {'mean': 121809.53333333334, 'std': 43467.796975357625, 'mode': 60000.0}, 'Po': {'mean': 64000.0, 'std': 4242.640687119285, 'mode': 64000.0}}


AttributeError: 'str' object has no attribute 'isnull'

In [48]:
print(list(train['BsmtQual'].unique()).remove(3))

ValueError: list.remove(x): x not in list

In [224]:
from sklearn.neighbors import KNeighborsClassifier


"""
    Divides the DataFrame into two parts:
    with na's and without na's
"""
def divideByNA(feature):
    X, y, X_test = [], [], []
    
    l = getNaIndexes(feature)
    for i in range(0, train.shape[0]):
        if not(i in l):
            X.append(train.iloc[i][feature])
            y.append(train.iloc[i][y_feature])
        else:
            X_test.append(0)
            
    return np.reshape(X, (-1, 1)),  np.reshape(y, (-1, 1)),  np.reshape(X_test, (-1, 1))
"""
    Fill the nas with knn methods
"""
def fillNaWithKNN(feature):
    knn = KNeighborsClassifier()
    
    X, y, X_test = divideByNA(feature)
    
    knn.fit(X, y)
    y = knn.predict(X_test)
    
    print(y)
    print('--------------')
    
for cat in missing_numerical_data:
    fillNaWithKNN(cat)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000 83000
 83000 83000 83000 83000 83000 83000 83000 83000 83

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


[147500 147500 147500 147500 147500 147500 147500 147500]
--------------
[132000 132000 132000 132000 132000 132000 132000 132000 132000 132000
 132000 132000 132000 132000 132000 132000 132000 132000 132000 132000
 132000 132000 132000 132000 132000 132000 132000 132000 132000 132000
 132000 132000 132000 132000 132000 132000 132000 132000 132000 132000
 132000 132000 132000 132000 132000 132000 132000 132000 132000 132000
 132000 132000 132000 132000 132000 132000 132000 132000 132000 132000
 132000 132000 132000 132000 132000 132000 132000 132000 132000 132000
 132000 132000 132000 132000 132000 132000 132000 132000 132000 132000
 132000]
--------------


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [229]:
m = train['MasVnrArea'].mean()
std = train['MasVnrArea'].std()

print((196-m)/std)

0.5098396881007375


In [5]:
train.corr()['SalePrice'].nlargest(21).keys()[1:]

Index(['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
       '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd',
       'GarageYrBlt', 'MasVnrArea', 'Fireplaces', 'BsmtFinSF1', 'LotFrontage',
       'WoodDeckSF', '2ndFlrSF', 'OpenPorchSF', 'HalfBath', 'LotArea'],
      dtype='object')

In [ ]:
output = pd.DataFrame({'Id': test_data.Id,
                      'SalePrice': pred})
output.to_csv('submission.csv', index=False)